In [58]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [59]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [61]:
# pip install sentence_transformers

In [62]:
# pip install textblob

In [63]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

import nltk
from nltk.tokenize import RegexpTokenizer       
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4") 

client=pymongo.MongoClient("192.168.10.240",27017)
db= client['healthcare_sideproject']
FHP_article = db['PMC_FHP_data']
FHP_article = FHP_article.find()
FHP_article = pd.DataFrame(FHP_article)
FHP_article = FHP_article.loc[:,'title':]
FHP_article

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,title,artical_date,abstract_list
0,Influence of forward head posture on muscle activation pattern of the trapezius pars descendens muscle in young adults,Published online 2022 Nov 14.,"Go to:\nAbstract\nForward head posture (FHP) is a serious problem causing head and neck disability, but the characteristics of muscle activity during long-term postural maintenance are unclear. This study aimed to investigate a comparison of electromyography (EMG) activation properties and subjective fatigue between young adults with and without habitual FHP. In this study, we examined the changes in the spatial and temporal distribution patterns of muscle activity using high-density surface EMG (HD-SEMG) in addition to mean frequency, a conventional measure of muscle fatigue. Nineteen male participants were included in the study (FHP group (n = 9; age = 22.3 ± 1.5 years) and normal group (n = 10; age = 22.5 ± 1.4 years)). Participants held three head positions (e.g., forward, backward, and neutral positions) for a total of 30 min each, and the EMG activity of the trapezius pars descendens muscle during posture maintenance was measured by HD-SEMG. The root mean square (RMS), the modified entropy, and the correlation coefficient were calculated. Additionally, the visual analogue scale (VAS) was evaluated to assess subjective fatigue. The RMS, VAS, modified entropy, and correlation coefficients were significantly higher in the FHP group than in the normal group (p < 0.001). With increasing postural maintenance time, the modified entropy and correlation coefficient values significantly decreased, and the mean frequency and VAS values significantly increased (p < 0.001). Furthermore, the forward position had significantly higher RMS, correlation coefficient, modified entropy, and VAS values than in the neutral position (p < 0.001). The HD-SEMG potential distribution patterns in the FHP group showed less heterogeneity and greater muscle activity in the entire muscle and subjective fatigue than those in the normal group. Excess muscle activity even in the neutral/comfortable position in the FHP group could potentially be a mechanism of neuromuscular conditions in this population.\nSubject terms: Health care, Engineering"
1,Relative posture between head and finger determines perceived tactile direction of motion,Published online 2020 Mar 26.,"Go to:\nAbstract\nThe hand explores the environment for obtaining tactile information that can be fruitfully integrated with other functions, such as vision, audition, and movement. In theory, somatosensory signals gathered by the hand are accurately mapped in the world-centered (allocentric) reference frame such that the multi-modal information signals, whether visual-tactile or motor-tactile, are perfectly aligned. However, an accumulating body of evidence indicates that the perceived tactile orientation or direction is inaccurate; yielding a surprisingly large perceptual bias. To investigate such perceptual bias, this study presented tactile motion stimuli to healthy adult participants in a variety of finger and head postures, and requested the participants to report the perceived direction of motion mapped on a video screen placed on the frontoparallel plane in front of the eyes. Experimental results showed that the perceptual bias could be divided into systematic and nonsystematic biases. Systematic bias, defined as the mean difference between the perceived and veridical directions, correlated linearly with the relative posture between the finger and the head. By contrast, nonsystematic bias, defined as minor difference in bias for different stimulus directions, was highly individualized, phase-locked to stimulus orientation presented on the skin. Overall, the present findings on systematic bias indicate that the transformation bias among the reference frames is dominated by the finger-to-head posture. Moreover, the highly individualized nature of nonsystematic bias reflects how information is obtained by th

## 데이터 전처리

### 결측치 처리 
- abstract가 없는 값은 제목으로 대체

In [64]:
FHP_article.query("abstract_list==''").count()

title            1790
artical_date     1790
abstract_list    1790
dtype: int64

In [65]:
FHP_article.loc[FHP_article.query("abstract_list==''").index,'abstract_list'] = FHP_article.loc[FHP_article.query("abstract_list==''").index,'title']

In [66]:
FHP_article.query("abstract_list==''")

,title,artical_date,abstract_list


### 중복값 제거

In [67]:
FHP_article.duplicated(subset='title').sum()

59

In [68]:
FHP_article = FHP_article.drop_duplicates(subset='title', keep='first', inplace=False, ignore_index=False)

### 발행일 변환

In [69]:
def date_format(raw):
    raw = raw.replace("Published online","")
    return raw

In [70]:
FHP_article.loc[:,'artical_date']=FHP_article['artical_date'].apply(date_format)


In [71]:
FHP_article

,title,artical_date,abstract_list
0,Influence of forward head posture on muscle activation pattern of the trapezius pars descendens muscle in young adults,2022 Nov 14.,"Go to:\nAbstract\nForward head posture (FHP) is a serious problem causing head and neck disability, but the characteristics of muscle activity during long-term postural maintenance are unclear. This study aimed to investigate a comparison of electromyography (EMG) activation properties and subjective fatigue between young adults with and without habitual FHP. In this study, we examined the changes in the spatial and temporal distribution patterns of muscle activity using high-density surface EMG (HD-SEMG) in addition to mean frequency, a conventional measure of muscle fatigue. Nineteen male participants were included in the study (FHP group (n = 9; age = 22.3 ± 1.5 years) and normal group (n = 10; age = 22.5 ± 1.4 years)). Participants held three head positions (e.g., forward, backward, and neutral positions) for a total of 30 min each, and the EMG activity of the trapezius pars descendens muscle during posture maintenance was measured by HD-SEMG. The root mean square (RMS), the modified entropy, and the correlation coefficient were calculated. Additionally, the visual analogue scale (VAS) was evaluated to assess subjective fatigue. The RMS, VAS, modified entropy, and correlation coefficients were significantly higher in the FHP group than in the normal group (p < 0.001). With increasing postural maintenance time, the modified entropy and correlation coefficient values significantly decreased, and the mean frequency and VAS values significantly increased (p < 0.001). Furthermore, the forward position had significantly higher RMS, correlation coefficient, modified entropy, and VAS values than in the neutral position (p < 0.001). The HD-SEMG potential distribution patterns in the FHP group showed less heterogeneity and greater muscle activity in the entire muscle and subjective fatigue than those in the normal group. Excess muscle activity even in the neutral/comfortable position in the FHP group could potentially be a mechanism of neuromuscular conditions in this population.\nSubject terms: Health care, Engineering"
1,Relative posture between head and finger determines perceived tactile direction of motion,2020 Mar 26.,"Go to:\nAbstract\nThe hand explores the environment for obtaining tactile information that can be fruitfully integrated with other functions, such as vision, audition, and movement. In theory, somatosensory signals gathered by the hand are accurately mapped in the world-centered (allocentric) reference frame such that the multi-modal information signals, whether visual-tactile or motor-tactile, are perfectly aligned. However, an accumulating body of evidence indicates that the perceived tactile orientation or direction is inaccurate; yielding a surprisingly large perceptual bias. To investigate such perceptual bias, this study presented tactile motion stimuli to healthy adult participants in a variety of finger and head postures, and requested the participants to report the perceived direction of motion mapped on a video screen placed on the frontoparallel plane in front of the eyes. Experimental results showed that the perceptual bias could be divided into systematic and nonsystematic biases. Systematic bias, defined as the mean difference between the perceived and veridical directions, correlated linearly with the relative posture between the finger and the head. By contrast, nonsystematic bias, defined as minor difference in bias for different stimulus directions, was highly individualized, phase-locked to stimulus orientation presented on the skin. Overall, the present findings on systematic bias indicate that the transformation bias among the reference frames is dominated by the finger-to-head posture. Moreover, the highly individualized nature of nonsystematic bias reflects how information is obtained by the orientation-selective units in t

### 정규화 : 대소문자 통합

In [72]:
def word_lower(raw):
    abstract_list = raw.split()
    modified_list = []
    for abstract_word in abstract_list :
         modified_list.append(abstract_word.lower())
    return " ".join(modified_list)

In [73]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_list'].apply(word_lower)

In [74]:
FHP_article.loc[2,'abstract_clean']

'go to: abstract instantaneous head posture (ihp) can extensively alter resting hyoid position in humans, yet postural effects on resting hyoid position remain poorly documented among mammals in general. clarifying this relationship is essential for evaluating interspecific variation in hyoid posture across evolution, and understanding its implications for hyolingual soft tissue function and swallowing motor control. using didelphis virginiana as a model, we conducted static manipulation experiments to show that head flexion shifts hyoid position rostrally relative to the cranium across different gapes. ihp-induced shifts in hyoid position along the anteroposterior axis are comparable to in vivo hyoid protraction distance during swallowing. ihp also has opposite effects on passive genio- and stylohyoid muscle lengths. high-speed biplanar videoradiography suggests didelphis consistently swallows at neutral to flexed posture, with stereotyped hyoid kinematics across different head postur

### 특수문자, 숫자, '/n', '소제목' 제거

In [75]:
import re
def word_cleaning(raw):
    raw = raw.replace("\n"," ")
    raw = raw.replace("go to: abstract","")
    raw = raw.replace("background:","")
    raw = raw.replace("purpose:","")
    raw = raw.replace("objectives:","")
    raw = raw.replace("objective:","")
    raw = raw.replace("design:","")
    raw = raw.replace("methods:","")
    raw = raw.replace("method:","")
    raw = raw.replace("results:","")
    raw = raw.replace("result:","")
    raw = raw.replace("discussion:","")
    raw = raw.replace("conclusions:","")
    raw = raw.replace("interpretation:","")
    raw = raw.replace("conclusion:","")
    raw = raw.replace("significance:","")
    raw = raw.replace("keywords:","")
    raw = raw.replace("fhp"," ")
    raw = raw.replace("forward head posture"," ")
    raw = raw.replace("head forward posture"," ")
    patterns = '([^a-z|A-Z|\'| ])'
    text_regex = re.sub(pattern = patterns, repl = "",string=raw)
    return text_regex

In [76]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_clean'].apply(word_cleaning)

In [77]:
FHP_article.loc[1789,'abstract_clean']

' background people with low back pain lbp exhibit impaired anticipatory postural adjustment apas objective to evaluate whether current motor retraining treatments address lbpassociated changes in movement coordination during tasks that do and do not require apas design prospectively registered randomized controlled trial with a blinded assessor setting outcome evaluations occurred in a university laboratory treatments in outpatient physical therapy clinics patients fifteen subjects without lbp and  subjects with chronic recurrent nonspecific lbp intervention twelve subjects with lbp received stabilization treatment  received movement system impairment msibased treatment over  weekly hour sessions plus home exercises measurements pre and post treatment surface electromyography emg was recorded bilaterally from trunk and leg muscles during unsupported and supported leglifting tasks which did and did not require an apa respectively vertical reaction forces under the contralateral leg wer

### 단어 토큰화

In [78]:
def word_tokenizing(raw):
    tokenizer = RegexpTokenizer("[\s]+", gaps=True) 
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS']]
    nouns_list = [t for t in tags if t[1] in  ['JJ','JJR','NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    return nouns_list

In [79]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_clean'].apply(word_tokenizing)

In [80]:
FHP_article['abstract_token']

0                                                                                                      [(is, VBZ), (serious, JJ), (problem, NN), (causing, VBG), (head, NN), (neck, NN), (disability, NN), (characteristics, NNS), (muscle, NN), (activity, NN), (longterm, JJ), (postural, JJ), (maintenance, NN), (are, VBP), (unclear, JJ), (study, NN), (aimed, VBD), (investigate, VB), (comparison, NN), (electromyography, NN), (emg, NN), (activation, NN), (properties, NNS), (subjective, JJ), (fatigue, NN), (young, JJ), (adults, NNS), (habitual, JJ), (study, NN), (examined, VBD), (changes, NNS), (spatial, JJ), (temporal, JJ), (distribution, NN), (patterns, NNS), (muscle, NN), (activity, NN), (using, VBG), (highdensity, NN), (surface, NN), (emg, NN), (hdsemg, NN), (addition, NN), (mean, VB), (frequency, NN), (conventional, JJ), (measure, NN), (muscle, NN), (fatigue, NN), (nineteen, JJ), (male, JJ), (participants, NNS), (were, VBD), (included, VBN), (study, NN), (group, NN), (n, JJ), (age, NN), (

### 표제어 추출

In [81]:
def Lemmatization(raw):
    lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    for word in raw:
        if word[1][0].lower() in ['v','n']:
            lemmatized_word = lemmatizer.lemmatize(word[0],word[1][0].lower())
            if lemmatized_word != 'be':
                lemmatized_list.append(lemmatized_word)
        else:
            lemmatized_list.append(word[0])
    return lemmatized_list

In [82]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_token'].apply(Lemmatization)

In [83]:
FHP_article.loc[:,'abstract_token']

0                                                                                     [serious, problem, cause, head, neck, disability, characteristic, muscle, activity, longterm, postural, maintenance, unclear, study, aim, investigate, comparison, electromyography, emg, activation, property, subjective, fatigue, young, adult, habitual, study, examine, change, spatial, temporal, distribution, pattern, muscle, activity, use, highdensity, surface, emg, hdsemg, addition, mean, frequency, conventional, measure, muscle, fatigue, nineteen, male, participant, include, study, group, n, age, year, normal, group, n, age, year, participant, hold, head, position, eg, neutral, position, total, min, emg, activity, trapezius, par, descendens, muscle, posture, maintenance, measure, hdsemg, root, mean, square, rms, modified, entropy, correlation, coefficient, calculate, visual, analogue, scale, va, evaluate, assess, subjective, fatigue, rms, vas, modify, ...]
1                  [hand, explore, environm

In [84]:
def join_words(raw):
    sentence = ' '.join(raw)
    return sentence

In [85]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_token'].apply(join_words)

In [86]:
FHP_article

,title,artical_date,abstract_list,abstract_clean,abstract_token,abstract_join
0,Influence of forward head posture on muscle activation pattern of the trapezius pars descendens muscle in young adults,2022 Nov 14.,"Go to:\nAbstract\nForward head posture (FHP) is a serious problem causing head and neck disability, but the characteristics of muscle activity during long-term postural maintenance are unclear. This study aimed to investigate a comparison of electromyography (EMG) activation properties and subjective fatigue between young adults with and without habitual FHP. In this study, we examined the changes in the spatial and temporal distribution patterns of muscle activity using high-density surface EMG (HD-SEMG) in addition to mean frequency, a conventional measure of muscle fatigue. Nineteen male participants were included in the study (FHP group (n = 9; age = 22.3 ± 1.5 years) and normal group (n = 10; age = 22.5 ± 1.4 years)). Participants held three head positions (e.g., forward, backward, and neutral positions) for a total of 30 min each, and the EMG activity of the trapezius pars descendens muscle during posture maintenance was measured by HD-SEMG. The root mean square (RMS), the modified entropy, and the correlation coefficient were calculated. Additionally, the visual analogue scale (VAS) was evaluated to assess subjective fatigue. The RMS, VAS, modified entropy, and correlation coefficients were significantly higher in the FHP group than in the normal group (p < 0.001). With increasing postural maintenance time, the modified entropy and correlation coefficient values significantly decreased, and the mean frequency and VAS values significantly increased (p < 0.001). Furthermore, the forward position had significantly higher RMS, correlation coefficient, modified entropy, and VAS values than in the neutral position (p < 0.001). The HD-SEMG potential distribution patterns in the FHP group showed less heterogeneity and greater muscle activity in the entire muscle and subjective fatigue than those in the normal group. Excess muscle activity even in the neutral/comfortable position in the FHP group could potentially be a mechanism of neuromuscular conditions in this population.\nSubject terms: Health care, Engineering",is a serious problem causing head and neck disability but the characteristics of muscle activity during longterm postural maintenance are unclear this study aimed to investigate a comparison of electromyography emg activation properties and subjective fatigue between young adults with and without habitual in this study we examined the changes in the spatial and temporal distribution patterns of muscle activity using highdensity surface emg hdsemg in addition to mean frequency a conventional measure of muscle fatigue nineteen male participants were included in the study group n age years and normal group n age years participants held three head positions eg forward backward and neutral positions for a total of min each and the emg activity of the trapezius pars descendens muscle during posture maintenance was measured by hdsemg the root mean square rms the modified entropy and the correlation coefficient were calculated additionally the visual analogue scale vas was evaluated to assess subjective fatigue the rms vas modified entropy and correlation coefficients were significantly higher in the group than in the normal group p with increasing postural maintenance time the modified entropy and correlation coefficient values significantly decreased and the mean frequency and vas values significantly increased p furthermore the forward position had significantly higher rms correlation coefficient modified entropy and vas values than in the neutral position p the hdsemg potential distribution patterns in the group showed less heterogeneity and greater muscle activity in the entire muscle and subjective fatigue than those in the normal group excess muscle activity even in the neutralcomfortable position in the gro

### 유사한 단어 묶기

In [87]:
def word_cleaning(raw):
    raw = raw.replace("chronic neck pain","cnp")
    raw = raw.replace("craniovertebral angle","cva")
    
    return raw

In [88]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_join'].apply(word_cleaning)

In [89]:
len(FHP_article)

12193

In [90]:
FHP_article.isnull().sum()

title             0
artical_date      0
abstract_list     0
abstract_clean    0
abstract_token    0
abstract_join     0
dtype: int64

## 키워드 분류하기

### TfidfVectorizer 최적화

In [155]:
from sklearn.feature_extraction.text import TfidfVectorizer
stop_word_list=[]
# stop_word_list = ['use','method','subject','compare','conclusion','suggest','have','show','result','study','include','provide','increase','investigate','significant','measure']
tfidfVectorizer = TfidfVectorizer(max_df=0.50,min_df=0.2,ngram_range=(1,2), stop_words = stop_word_list)
features = tfidfVectorizer.fit_transform(FHP_article['abstract_join'])
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['aim', 'analysis', 'body', 'change', 'compare', 'conclusion',
       'control', 'different', 'effect', 'group', 'have', 'include',
       'increase', 'measure', 'method', 'movement', 'patient', 'perform',
       'posture', 'provide', 'result', 'show', 'significant', 'system',
       'test'], dtype=object)

### gridsearch

In [156]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

# LDA 모델을 위한 파라미터 그리드 생성
param_grid = {
    'n_components': [3],
    'learning_decay': [0.7],
    'doc_topic_prior': [1.0],  # 문서-주제 사전확률 (알파)
    'topic_word_prior': [1.0]  # 단어-주제 사전확률 (베타)
}

# LatentDirichletAllocation 모델 인스턴스 생성
lda = LatentDirichletAllocation()

# GridSearchCV 인스턴스 생성
model = GridSearchCV(lda, param_grid, n_jobs=-1, cv=5)

# 데이터는 여기에 당신의 데이터셋을 사용하세요
# X는 문서-단어 행렬이어야 합니다. 예를 들어, CountVectorizer나 TfidfVectorizer를 사용하여 생성할 수 있습니다.
model.fit(features)

# 최적 모델
grid_best_model = model.best_estimator_

# 최적의 파라미터와 점수를 출력
print("Best Model's Params: ", model.best_params_)
print("Best Log Likelihood Score: ", model.best_score_)
print("perplexity",grid_best_model.perplexity(features) )


Best Model's Params:  {'doc_topic_prior': 1.0, 'learning_decay': 0.7, 'n_components': 3, 'topic_word_prior': 1.0}
Best Log Likelihood Score:  -16683.787496207508
perplexity 30.945672570794994


### 전체 데이터에 적용

In [163]:
# grid_best_model.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)

In [164]:
topics_output = grid_best_model.transform(features)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
0    5257
2    3528
1    3408
Name: count, dtype: int64

In [165]:
df_datas_0 = [grid_best_model.components_[0], dictionary_list]
df_topics_0 = pd.DataFrame(data=df_datas_0).T
df_topics_0 = df_topics_0.dropna()
df_topics_0 = df_topics_0.sort_values(0, ascending=False).reset_index()
df_topics_0.rename(columns = {1 : 'first_components'}, inplace = True)
df_topics_0.rename(columns = {0 : 'score'}, inplace = True)
df_datas_1 = [grid_best_model.components_[1], dictionary_list]
df_topics_1 = pd.DataFrame(data=df_datas_1).T
df_topics_1 = df_topics_1.dropna()
df_topics_1 = df_topics_1.sort_values(0, ascending=False).reset_index()
df_topics_1.rename(columns = {1 : 'second_components'}, inplace = True)
df_topics_1.rename(columns = {0 : 'score'}, inplace = True)
df_datas_2 = [grid_best_model.components_[2], dictionary_list]
df_topics_2 = pd.DataFrame(data=df_datas_2).T
df_topics_2 = df_topics_2.dropna()
df_topics_2 = df_topics_2.sort_values(0, ascending=False).reset_index()
df_topics_2.rename(columns = {1 : 'third_components'}, inplace = True)
df_topics_2.rename(columns = {0 : 'score'}, inplace = True)
df_topic = pd.concat([df_topics_0.loc[:20,['score','first_components']],df_topics_1.loc[:20,['score','second_components']],df_topics_2.loc[:20,['score','third_components']]],axis=1)
df_topic

,score,first_components,score,second_components,score,third_components
0,812.27514,movement,1287.568229,patient,946.653676,posture
1,668.949319,control,1019.982684,have,855.289806,body
2,635.946332,test,608.191305,provide,632.2245,method
3,619.995055,different,551.402745,control,539.046907,system
4,582.630841,analysis,499.345499,group,526.394804,include
5,582.012777,show,453.618346,effect,490.15561,group
6,557.696407,change,412.745486,result,452.75385,result
7,487.650665,increase,321.118939,system,443.677077,compare
8,424.346733,have,285.718409,significant,414.036996,measure
9,347.918309,effect,281.860201,perform,313.575714,conclusion


In [160]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [161]:
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
0    5257
2    3528
1    3408
Name: count, dtype: int64

In [162]:
df_topics_score['dominant_topic_number']

0        2
1        2
2        2
3        2
4        0
        ..
12188    0
12189    0
12190    0
12191    0
12192    1
Name: dominant_topic_number, Length: 12193, dtype: int64

## 토픽 분석하기

### topic별로 분류하기

In [110]:
FHP_article = FHP_article.reset_index()
FHP_article.loc[:,'article_topic'] = df_topics_score['dominant_topic_number']

In [111]:
FHP_article.isnull().sum()

index             0
title             0
artical_date      0
abstract_list     0
abstract_clean    0
abstract_token    0
abstract_join     0
article_topic     0
dtype: int64

### topic 별 많이 등장하는 단어 추출

In [112]:
FHP_article.query("article_topic == 0")

,index,title,artical_date,abstract_list,abstract_clean,abstract_token,abstract_join,article_topic
1,1,Relative posture between head and finger determines perceived tactile direction of motion,2020 Mar 26.,"Go to:\nAbstract\nThe hand explores the environment for obtaining tactile information that can be fruitfully integrated with other functions, such as vision, audition, and movement. In theory, somatosensory signals gathered by the hand are accurately mapped in the world-centered (allocentric) reference frame such that the multi-modal information signals, whether visual-tactile or motor-tactile, are perfectly aligned. However, an accumulating body of evidence indicates that the perceived tactile orientation or direction is inaccurate; yielding a surprisingly large perceptual bias. To investigate such perceptual bias, this study presented tactile motion stimuli to healthy adult participants in a variety of finger and head postures, and requested the participants to report the perceived direction of motion mapped on a video screen placed on the frontoparallel plane in front of the eyes. Experimental results showed that the perceptual bias could be divided into systematic and nonsystematic biases. Systematic bias, defined as the mean difference between the perceived and veridical directions, correlated linearly with the relative posture between the finger and the head. By contrast, nonsystematic bias, defined as minor difference in bias for different stimulus directions, was highly individualized, phase-locked to stimulus orientation presented on the skin. Overall, the present findings on systematic bias indicate that the transformation bias among the reference frames is dominated by the finger-to-head posture. Moreover, the highly individualized nature of nonsystematic bias reflects how information is obtained by the orientation-selective units in the S1 cortex.\nSubject terms: Somatosensory system, Human behaviour",the hand explores the environment for obtaining tactile information that can be fruitfully integrated with other functions such as vision audition and movement in theory somatosensory signals gathered by the hand are accurately mapped in the worldcentered allocentric reference frame such that the multimodal information signals whether visualtactile or motortactile are perfectly aligned however an accumulating body of evidence indicates that the perceived tactile orientation or direction is inaccurate yielding a surprisingly large perceptual bias to investigate such perceptual bias this study presented tactile motion stimuli to healthy adult participants in a variety of finger and head postures and requested the participants to report the perceived direction of motion mapped on a video screen placed on the frontoparallel plane in front of the eyes experimental results showed that the perceptual bias could be divided into systematic and nonsystematic biases systematic bias defined as the mean difference between the perceived and veridical directions correlated linearly with the relative posture between the finger and the head by contrast nonsystematic bias defined as minor difference in bias for different stimulus directions was highly individualized phaselocked to stimulus orientation presented on the skin overall the present findings on systematic bias indicate that the transformation bias among the reference frames is dominated by the fingertohead posture moreover the highly individualized nature of nonsystematic bias reflects how information is obtained by the orientationselective units in the s cortex subject terms somatosensory system human behaviour,"[hand, explore, environment, obtain, tactile, information, integrate, other, function, such, vision, audition, movement, theory, somatosensory, signal, gather, hand, map, worldcentered, allocentric, reference, frame, such, multimodal, information, signal, visualtactile, motortactile, align, accumulate, body, evidence, indicate, perceive, tactile, orientation, direc

In [113]:
len(FHP_article.query("article_topic == 1"))

3456

In [114]:
FHP_article.query("article_topic == 2")

,index,title,artical_date,abstract_list,abstract_clean,abstract_token,abstract_join,article_topic
4,4,Changes in deep neck muscle length from the neutral to forward head posture. A cadaveric study using Thiel cadavers,2022 Jan 25.,"Go to:\nAbstract\nForward head posture (FHP) is one of the most common postural deviations. Deep neck muscle imbalance of individuals with FHP is of primary concern in clinical rehabilitation. However, there is scarce quantitative research on changes in deep neck muscle length with the head moving forward. This study aimed to investigate changes in deep neck muscle length with different severity levels of FHP. Six Thiel‐embalmed cadavers (four males and two females) were dissected, and 16 deep neck muscles in each cadaver were modeled by a MicroScribe 3D Digitizer in the neutral head posture, slight FHP, and severe FHP. The craniovertebral angle was used to evaluate the degrees of FHP. Quantitative length change of the deep neck muscles was analyzed using Rhinoceros 3D. In slight FHP significant changes in length occurred in four muscles: two shortened (upper semispinalis capitis, rectus capitis posterior minor) and two lengthened (longus capitis, splenius cervicis). In severe FHP all occipital extensors were significantly shortened (10.6 ± 6.4%), except for obliquus capitis superior, and all cervical extensors were significantly lengthened (4.8 ± 3.4%), while longus capitis (occipital flexor) and the superior oblique part of the longus colli (cervical flexor) were lengthened by 8.8 ± 3.8% and 4.2 ± 3.1%, respectively. No significant length change was observed for the axial rotator. This study presents an alternate anatomical insight into the clinical rehabilitation of FHP. Six muscles appear to be important in restoring optimal head posture, with improvements in FHP being related to interventions associated with the occipital and cervical extensors.\nKeywords: cervical spine, deep neck muscles, forward head posture, muscle length change",is one of the most common postural deviations deep neck muscle imbalance of individuals with is of primary concern in clinical rehabilitation however there is scarce quantitative research on changes in deep neck muscle length with the head moving forward this study aimed to investigate changes in deep neck muscle length with different severity levels of six thielembalmed cadavers four males and two females were dissected and deep neck muscles in each cadaver were modeled by a microscribe d digitizer in the neutral head posture slight and severe the craniovertebral angle was used to evaluate the degrees of quantitative length change of the deep neck muscles was analyzed using rhinoceros d in slight significant changes in length occurred in four muscles two shortened upper semispinalis capitis rectus capitis posterior minor and two lengthened longus capitis splenius cervicis in severe all occipital extensors were significantly shortened except for obliquus capitis superior and all cervical extensors were significantly lengthened while longus capitis occipital flexor and the superior oblique part of the longus colli cervical flexor were lengthened by and respectively no significant length change was observed for the axial rotator this study presents an alternate anatomical insight into the clinical rehabilitation of six muscles appear to be important in restoring optimal head posture with improvements in being related to interventions associated with the occipital and cervical extensors cervical spine deep neck muscles muscle length change,"[common, postural, deviation, deep, neck, muscle, imbalance, individual, primary, concern, clinical, rehabilitation, scarce, quantitative, research, change, deep, neck, muscle, length, head, move, study, aim, investigate, change, deep, neck, muscle, length, different, severity, level, thielembalmed, cadaver, male, female, dissect, deep, neck, muscle, cadaver, model, microscribe, d, digitizer, neutral, head, posture, slight, severe,

In [115]:
FHP_article.to_csv('FHP_article.csv',index=False,encoding='utf-8')

## 결론
- 논문
- 중복 논문을 삭제하고, 결측치를 제거하니 데이터가 1000개정도밖에 남지 않음
- 다른 사이트로 다시 스크래핑 후 재시도

### Count vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words=[], ngram_range=(1, 2))

In [ ]:
FHP_article.query("article_topic == 2")['abstract_join']

In [ ]:
count_vect.fit(FHP_article['abstract_join'])

In [ ]:
count_vect.vocabulary_

In [ ]:
dtm_topic_0 = count_vect.transform(FHP_article.query("article_topic == 0")['abstract_join'])
dtm_array_0 = dtm_topic_0.toarray()
df_dtm_0 = pd.DataFrame(dtm_array_0, columns = vocabs)
df_first_topic = df_dtm_0.sum().sort_values(ascending=False)[:20]
df_first_topic = df_first_topic.reset_index()
df_first_topic.rename(columns = {'index' : 'first'}, inplace = True)
df_first_topic.rename(columns = {0 : 'count'}, inplace = True)

In [ ]:
dtm_topic_1 = count_vect.transform(FHP_article.query("article_topic == 1")['abstract_join'])
dtm_array_1 = dtm_topic_1.toarray()
df_dtm_1 = pd.DataFrame(dtm_array_1, columns = vocabs)
df_second_topic = df_dtm_1.sum().sort_values(ascending=False)[:20]
df_second_topic = df_second_topic.reset_index()
df_second_topic.rename(columns = {'index' : 'second'}, inplace = True)
df_second_topic.rename(columns = {0 : 'count'}, inplace = True)

In [ ]:
dtm_topic_2 = count_vect.transform(FHP_article.query("article_topic == 2")['abstract_join'])
dtm_array_2 = dtm_topic_2.toarray()
df_dtm_2 = pd.DataFrame(dtm_array_2, columns = vocabs)
df_third_topic = df_dtm_2.sum().sort_values(ascending=False)[:20]
df_third_topic = df_third_topic.reset_index()
df_third_topic.rename(columns = {'index' : 'third'}, inplace = True)
df_third_topic.rename(columns = {0 : 'count'}, inplace = True)

In [ ]:
vocabs = count_vect.get_feature_names_out()
vocabs[:2]
vocabs.shape

In [ ]:
df_topic = pd.concat([df_first_topic,df_second_topic,df_third_topic],axis=1)

In [ ]:
df_topic